In [1]:
from src import api as API

API.DataAPI.update()
API.DataAPI.prepare_train_data()
API.Predictor.update_factors()

Update Files
Sat Jun 29 09:04:28 2024 : Updated ~ data\DataBase\DB_information\calendar.feather Done! Cost 0.02 Secs
Sat Jun 29 09:04:28 2024 : Updated ~ data\DataBase\DB_information\description.feather Done! Cost 0.03 Secs
Sat Jun 29 09:04:28 2024 : Updated ~ data\DataBase\DB_information\st.feather Done! Cost 0.03 Secs
Sat Jun 29 09:04:33 2024 : Updated ~ data\DataBase\DB_information\industry.feather Done! Cost 5.03 Secs
Sat Jun 29 09:04:41 2024 : Updated ~ data\DataBase\DB_information\concepts.feather Done! Cost 7.80 Secs
Sat Jun 29 09:04:44 2024 : Updated ~ data\DataBase\DB_models\risk_exp\2024\risk_exp.20240628.feather Done! Cost 0.08 Secs
Sat Jun 29 09:04:44 2024 : Updated ~ data\DataBase\DB_models\risk_cov\2024\risk_cov.20240628.feather Done! Cost 0.07 Secs
Sat Jun 29 09:04:44 2024 : Updated ~ data\DataBase\DB_models\risk_spec\2024\risk_spec.20240628.feather Done! Cost 0.03 Secs
Sat Jun 29 09:04:51 2024 : Updated ~ data\DataBase\DB_labels\ret20_lag\2024\ret20_lag.20240529.feather

RuntimeError: Error(s) in loading state_dict for gru:
	Unexpected key(s) in state_dict: "multiloss_alpha". 

In [1]:
from src import api as API
API.Predictor.update_factors()

Finish model [gru_day] predicting!
Finish model [gruRTN_day] predicting!
Finish model [gruRES_day] predicting!
--------------------------------------------------------------------------------


In [2]:
from src import api
api.ModelTestor.new('ts_mixer').try_metrics()

Load  2 DataBlocks ...

... cost 0.22 secs
Align 2 DataBlocks ...... cost 0.21 secs
Pre-Norming method of [day] : {'divlast': True, 'histnorm': True}
x shape is torch.Size([5064, 30, 6])
y shape is torch.Size([5064, 1])
Test Forward Success
metrics :  Metrics.MetricOutput(loss=tensor(1.0688, grad_fn=<AddBackward0>), score=-0.015420470386743546, loss_item=1.068833589553833, penalty=0.0, losses=tensor(1.0688, grad_fn=<ExpBackward0>))
Test Metrics Success
